In [1]:
using DataFrames
using CSV

# Purpose
This module is designed for reading in marker and dye data file.

The data shall be sotred in ".csv" format.

## Dyes
### dye_excitation.csv
spectral data of excitation of each dye, from 300 to 999 nm (step 1 nm).

### dye_emission.csv
spectral data of emission of each dye, from 300 to 999 nm (step 1 nm).

## Markers
### antigens.csv
first 